In [131]:
page_store = set()

In [1]:
import pandas as pd

aug = pd.read_csv('august2023.csv')
sept = pd.read_csv('september2023.csv')
octo = pd.read_csv('october2023.csv')

In [186]:
page = 'September_11_attacks'

def get_max_spike_page(page, central_df, prev_month_df, next_month_df):
    pages_to_check = set([*central_df[(central_df['prev']==page) & (central_df['clicks']>=100)]['curr'].values,
                        *central_df[(central_df['curr']==page) & (central_df['clicks']>=100)]['prev'].values,
                        *prev_month_df[(prev_month_df['prev']==page) & (prev_month_df['clicks']>=100)]['curr'].values,
                        *prev_month_df[(prev_month_df['curr']==page) & (prev_month_df['clicks']>=100)]['prev'].values,
                        *next_month_df[(next_month_df['prev']==page) & (next_month_df['clicks']>=130)]['curr'].values,
                        *next_month_df[(next_month_df['curr']==page) & (next_month_df['clicks']>=130)]['prev'].values,
                        page
                        ])
    inflow_prev = prev_month_df[prev_month_df['type']!='other'].groupby('curr')[['clicks']].agg(sum)
    inflow_central = central_df[central_df['type']!='other'].groupby('curr')[['clicks']].agg(sum)
    inflow_next = next_month_df[next_month_df['type']!='other'].groupby('curr')[['clicks']].agg(sum)

    inflow_prev = inflow_prev[inflow_prev.index.isin(pages_to_check)]
    inflow_central = inflow_central[inflow_central.index.isin(pages_to_check)]
    inflow_next = inflow_next[inflow_next.index.isin(pages_to_check)]

    merged = inflow_central.join(inflow_prev, lsuffix='_central', rsuffix='_prev')
    merged = merged.join(inflow_next, rsuffix='_next')
    merged['clicks_next'] = merged['clicks']
    merged=merged.drop('clicks',axis=1)
    merged['spike_ratio'] = (2*merged['clicks_central'])/(merged['clicks_prev']+merged['clicks_next'])
    next_step_pages = merged[merged['spike_ratio']>1.5].sort_values('clicks_central', ascending=False)
    return next_step_pages.head()

get_max_spike_page(page=page, central_df=sept, prev_month_df=aug, next_month_df=octo)


,clicks_central,clicks_prev,clicks_next,spike_ratio
curr,,,,
September_11_attacks,1950667,470818.0,492785,4.048694
Dianne_Feinstein,1175875,259820.0,515358,3.033819
Aaron_Rodgers,931974,417795.0,262737,2.738957
Mark_Wahlberg,584859,341908.0,300941,1.819584
World_Trade_Center_(1973–2001),532825,163411.0,186482,3.045645


In [193]:
def get_spiked_pages(page, central_df, prev_month_df, next_month_df):
    pages_to_check = set([*central_df[(central_df['prev']==page) & (central_df['clicks']>=100)]['curr'].values,
                        #*central_df[(central_df['curr']==page) & (central_df['clicks']>=100)]['prev'].values,
                        *prev_month_df[(prev_month_df['prev']==page) & (prev_month_df['clicks']>=100)]['curr'].values,
                        #*prev_month_df[(prev_month_df['curr']==page) & (prev_month_df['clicks']>=100)]['prev'].values,
                        *next_month_df[(next_month_df['prev']==page) & (next_month_df['clicks']>=130)]['curr'].values,
                        #*next_month_df[(next_month_df['curr']==page) & (next_month_df['clicks']>=130)]['prev'].values,
                        #page
                        ])
    inflow_prev = prev_month_df[prev_month_df['type']=='link'].groupby('curr')[['clicks']].agg(sum)
    inflow_central = central_df[central_df['type']=='link'].groupby('curr')[['clicks']].agg(sum)
    inflow_next = next_month_df[next_month_df['type']=='link'].groupby('curr')[['clicks']].agg(sum)

    inflow_prev = inflow_prev[inflow_prev.index.isin(pages_to_check)]
    inflow_central = inflow_central[inflow_central.index.isin(pages_to_check)]
    inflow_next = inflow_next[inflow_next.index.isin(pages_to_check)]

    merged = inflow_central.join(inflow_prev, lsuffix='_central', rsuffix='_prev')
    merged = merged.join(inflow_next, rsuffix='_next')
    merged['clicks_next'] = merged['clicks']
    merged=merged.drop('clicks',axis=1)
    merged['spike_ratio'] = (2*merged['clicks_central'])/(merged['clicks_prev']+merged['clicks_next'])
    next_step_pages = merged[merged['spike_ratio']>1.5].sort_values('clicks_central', ascending=False)
    return next_step_pages.index.values

queue = [page]
visited_pages = set()

level_size = 1
next_level_size = 0

spike_level_nums = []

while len(queue)!=0:
    node = queue.pop(0)
    level_size-=1
    if node not in visited_pages:
        new_pages = get_spiked_pages(page, central_df=sept, prev_month_df=aug, next_month_df=octo)
        next_level_size += len(new_pages)
        queue.extend(new_pages)
        visited_pages.add(node)
    if level_size==0:
        spike_level_nums.append(next_level_size)
        print(spike_level_nums)
        level_size = next_level_size
        next_level_size = 0
    

[194]
[194, 37636]
[194, 37636, 0]


In [194]:
len(visited_pages)

195

In [180]:
merged['spike_ratio'] = (2*merged['clicks_sept'])/(merged['clicks_aug']+merged['clicks_oct'])